Script: clean_presidential_elections.ipynb
Author: Michel Gutmann
Purpose: Generates a state-election year dataset with presidential election results by party from David Leip's dataset.

In [5]:
import pandas as pd
import us
import os
import os.path as path
import numpy as np

In [6]:
parent_dir = os.path.abspath("/Users/revekkagershovich/Dropbox (MIT)/StateLaws")
os.chdir(parent_dir)
assert os.path.exists(parent_dir), "parent_dir does not exist"
data_dir = "./2_data/2_intermediate/political_data"
assert os.path.exists(data_dir), "Data directory does not exist"

In [7]:
input_file = "./2_data/1_raw/political_data/StateLevelData.xlsx"
assert os.path.exists(input_file), "Input file does not exist"
output_file = os.path.join(data_dir, "./statelevel_preselection_results.csv")
assert os.path.exists(output_file), "Output file does not exist"



In [32]:
##### DEFINE MAIN CLEANING FUNCTION #####
def clean_election(df):
    # Standardize format by keeping only raw result columns
    i = 0
    cols_to_keep = ["Unnamed: 0"]
    for col in df.columns:
        if i == 1:
            cols_to_keep.append(col)
        elif col == "% Total Vote":
            i = 1
    cols_to_keep = cols_to_keep[:-2]
    df_filtered = df[cols_to_keep].copy()
    df_filtered.rename({"Unnamed: 0": "state"}, axis=1, inplace=True)

    # Rename columns to appropriate names
    col_names = ["state"]
    for col in df_filtered.columns:
        col_str = str(col)
        if col_str != "state" and not col_str.startswith("Unnamed: "):
            last_name = "".join(col_str.lower().split(" "))
            if last_name not in ["democratic", "republican"]:
                last_name = "thirdparties"
            elif last_name == "democratic":
                last_name = "dem"
            elif last_name == "republican":
                last_name = "rep"
            col_names.append("votes_" + last_name)
        elif col_str != "state":
            col_names.append("pct_" + last_name)
    df_filtered.columns = col_names
    # df_filtered1 = df_filtered.loc[:, (df_filtered.dtypes != 'float64')]
    # with pd.option_context('display.max_rows', None):
    #     print(df_filtered1.dtypes)
    #     print(df_filtered1.head())
    # df_filtered2 = df_filtered.iloc[:, 3:7]
    # with pd.option_context('display.max_rows', None):
    #     # print(df_filtered2.dtypes)
    #     print(df_filtered2.head())
    # Keep only float type columns and 'state' column
    df_filtered = df_filtered.loc[:, (df_filtered.dtypes == 'float64') | (df_filtered.columns == 'state')]
     
    # Sum over third party results
    cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
    return cleaned_df


In [9]:
##### LOAD DATA AND PUT IN FULL DATASET #####
# Load the Excel file with multiple sheets
dfs = pd.read_excel(input_file, header=1, sheet_name=None)

/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/

In [55]:
df_2008 = dfs["2008"]
# df_2008 = df_2008.loc[:, ['State', 'Unnamed: 63']]
# print(df_2008.head())
df_2008 = df_2008.drop(['State', 'Unnamed: 63'], axis=1)

In [56]:
with pd.option_context('display.max_columns', None):
    print(df_2008.head())

   Unnamed: 0  Elec Vote Popular Vote     D     R   O  D.1  R.1  I      Votes  \
0     Alabama        9.0      2099819   NaN   9.0 NaN  2.0  1.0  3   453067.0   
1      Alaska        3.0       326197   NaN   3.0 NaN  2.0  1.0  3    70247.0   
2     Arizona       10.0      2303838   NaN  10.0 NaN  2.0  1.0  4   195404.0   
3    Arkansas        6.0      1086617   NaN   6.0 NaN  2.0  1.0  3   215707.0   
4  California       55.0     13577265  55.0   NaN NaN  1.0  2.0  3  3262692.0   

   % Total Vote  Democratic  Unnamed: 12  Republican  Unnamed: 14  \
0      0.215765    813479.0     0.387404   1266546.0     0.603169   
1      0.215351    123594.0     0.378894    193841.0     0.594245   
2      0.084817   1034707.0     0.449123   1230111.0     0.533940   
3      0.198512    422310.0     0.388647    638017.0     0.587159   
4      0.240306   8274473.0     0.609436   5011781.0     0.369130   

   Independent  Unnamed: 16  Libertarian  Unnamed: 18    Green   Unnamed: 20  \
0       6788.0    

In [29]:
df_2008 = dfs["2008"]
df_2008_clean = clean_election(df_2008)

state                 object
votes_thirdparties    object
pct_thirdparties      object
dtype: object
        state votes_thirdparties pct_thirdparties
0     Alabama            Alabama               AL
1      Alaska             Alaska               AK
2     Arizona            Arizona               AZ
3    Arkansas           Arkansas               AR
4  California         California               CA
   votes_rep   pct_rep  votes_thirdparties  pct_thirdparties
0  1266546.0  0.603169              6788.0          0.003233
1   193841.0  0.594245              3783.0          0.011597
2  1230111.0  0.533940             11301.0          0.004905
3   638017.0  0.587159             12882.0          0.011855
4  5011781.0  0.369130            108381.0          0.007983


In [11]:
df_2008_clean.head()

,pct_dem,pct_rep,pct_thirdparties,state,votes_dem,votes_rep,votes_thirdparties
0,0.387404,0.603169,34.009427,Alabama,813479.0,1266546.0,19804.0
1,0.378894,0.594245,37.026861,Alaska,123594.0,193841.0,8767.0
2,0.449123,0.533940,35.016937,Arizona,1034707.0,1230111.0,46658.0
3,0.388647,0.587159,28.024194,Arkansas,422310.0,638017.0,34949.0
4,0.609436,0.369130,27.021434,California,8274473.0,5011781.0,303098.0


In [36]:
print(19804/(813479+1266546))

0.009521039410584008


In [33]:
df_2012 = dfs["2012"]
df_2012_clean = clean_election(df_2012)

/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/4012308652.py:43: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()


In [34]:
df_2012_clean.head()

,pct_dem,pct_rep,pct_thirdparties,state,votes_dem,votes_rep,votes_thirdparties
0,0.383590,0.605458,0.010951,Alabama,795696.0,1255925.0,22717.0
1,0.408127,0.548016,0.043858,Alaska,122640.0,164676.0,13179.0
2,0.444485,0.534846,0.020662,Arizona,1025232.0,1233654.0,47659.0
3,0.368790,0.605669,0.025541,Arkansas,394409.0,647744.0,27315.0
4,0.602390,0.371204,0.026407,California,7854285.0,4839958.0,344304.0


In [12]:
# Process each sheet (year) and combine them into a single DataFrame
i = 0
for key in dfs.keys():
    if key != "Copyright" and i == 0:
        print(f"Processing {key}")
        new_df = clean_election(dfs[key])
        new_df["year"] = int(key)
        full_df = new_df
        i += 1
    if key == 2008:
        print(f"Processing {key}")
        df_2008 = dfs[key].drop(['State', 'Unnamed: 63'], axis=1)
        new_df = clean_election(df_2008)
        new_df["year"] = int(key)
        full_df = pd.concat([full_df, new_df], verify_integrity=True, axis=0, ignore_index=True)
    elif key != "Copyright":
        print(f"Processing {key}")
        new_df = clean_election(dfs[key])
        new_df["year"] = int(key)
        full_df = pd.concat([full_df, new_df], verify_integrity=True, axis=0, ignore_index=True)


/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered

Processing 2012
state                  object
votes_dem             float64
pct_dem               float64
votes_rep             float64
pct_rep               float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      float64
votes_thirdparties    float64
pct_thirdparties      fl

/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2161601794.py:37: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered

In [13]:
##### PREPARE AND OUTPUT FULL DATASET #####
# Remove asterisks from state names
full_df["state"] = full_df["state"].apply(lambda x: str(x)[:-1] if str(x).endswith("*") else str(x)) 

# Only keep rows corresponding to actual U.S. states
full_df = full_df.loc[
    full_df["state"].isin([state.name for state in us.STATES])
].copy()

# Add FIPS codes to the dataset
state_map = us.states.mapping("name", "fips")
full_df["statefips"] = full_df["state"].apply(lambda x: state_map[x])

# Save the cleaned dataset to the output file
# full_df.to_csv(output_file, index=False)
print(f"Dataset saved to {output_file}")

Dataset saved to ./2_data/2_intermediate/political_data/./statelevel_preselection_results.csv
